In [14]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [15]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [19]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [20]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t"))

In [22]:
rdd_film = sc.textFile("/labs/laba01/ml-100k/u.item").map(lambda x: x.split("|"))

user_id, item_id, rating, timestamp

In [24]:
rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

movie_id, ...

In [46]:
rdd_film.take(1)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [27]:
rdd.groupBy(lambda x: x[2]).collect()

[('1', <pyspark.resultiterable.ResultIterable at 0x7f6c25812400>),
 ('4', <pyspark.resultiterable.ResultIterable at 0x7f6c2543b320>),
 ('3', <pyspark.resultiterable.ResultIterable at 0x7f6c25812748>),
 ('2', <pyspark.resultiterable.ResultIterable at 0x7f6c1b8844a8>),
 ('5', <pyspark.resultiterable.ResultIterable at 0x7f6c1b884470>)]

In [50]:
count_ratings_all = rdd.map(lambda x: (x[2], 1)).groupByKey().sortByKey().map(lambda x: (x[0], len(x[1]))).map(lambda x: x[1]).collect()

In [51]:
count_ratings_all

[6110, 11370, 27145, 34174, 21201]

In [54]:
count_ratings_328 = rdd.filter(lambda x: x[1] == '328').map(lambda x: (x[2], 1)).groupByKey().sortByKey().map(lambda x: (x[0], len(x[1]))).map(lambda x: x[1]).collect()

In [55]:
count_ratings_328

[12, 40, 94, 109, 40]

In [56]:
result = {}
result['hist_film'] = count_ratings_328
result['hist_all'] = count_ratings_all

In [57]:
result

{'hist_film': [12, 40, 94, 109, 40],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [58]:
import json

with open("lab01.json", "w") as outfile:
    json.dump(result, outfile)

In [59]:
spark.stop()